# Activity: Validate and clean your data

## Introduction

In this activity, you will use input validation and label encoding to prepare a dataset for analysis. These are fundamental techniques used in all types of data analysis, from simple linear regression to complex neural networks. 

In this activity, you are a data professional an investment firm that is attempting to invest in private companies with a valuation of at least $1 billion. These are often known as "unicorns." Your client wants to develop a better understanding of unicorns, with the hope they can be early investors in future highly successful companies. They are particularly interested in the investment strategies of the three top unicorn investors: Sequoia Capital, Tiger Global Management, and Accel. 

## Step 1: Imports

In [6]:
# Import libraries and packages.

import pandas as pd
import seaborn as sns
from matplotlib import pyplot

### Load the dataset

In [8]:
# display all columns
pd.set_option('display.max_columns', None)

In [9]:
# IMPORT YOUR DATA. 
companies = pd.read_csv(r"C:\Users\user\Downloads\Modified_Unicorn_Companies.csv")

# Display the first five rows.
companies[:5]

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors
0,Bytedance,180,2017-04-07,Artificial intelligence,Beijing,China,Asia,2012,$8B,"Sequoia Capital China, SIG Asia Investments, S..."
1,SpaceX,100,2012-12-01,Other,Hawthorne,United States,North America,2002,$7B,"Founders Fund, Draper Fisher Jurvetson, Rothen..."
2,SHEIN,100,2018-07-03,E-commerce & direct-to-consumer,Shenzhen,China,Asia,2008,$2B,"Tiger Global Management, Sequoia Capital China..."
3,Stripe,95,2014-01-23,FinTech,San Francisco,United States,North America,2010,$2B,"Khosla Ventures, LowercaseCapital, capitalG"
4,Klarna,46,2011-12-12,Fintech,Stockholm,Sweden,Europe,2005,$4B,"Institutional Venture Partners, Sequoia Capita..."


## Step 2: Data cleaning


Begin by displaying the data types of the columns in `companies`.

In [12]:
# Display the data types of the columns.

companies.dtypes

Company             object
Valuation            int64
Date Joined         object
Industry            object
City                object
Country/Region      object
Continent           object
Year Founded         int64
Funding             object
Select Investors    object
dtype: object

### Modify the data types

Notice that the data type of the `Date Joined` column is an `object`&mdash;in this case, a string. Convert this column to `datetime` to make it more usable. 

In [15]:
# Apply necessary datatype conversions.

companies['Date Joined'] = pd.to_datetime(companies['Date Joined'])

### Create a new column

Add a column called `Years To Unicorn`, which is the number of years between when the company was founded and when it became a unicorn.

In [18]:
# Create the column Years To Unicorn.

companies['Years to Unicorn'] = (companies['Date Joined']).dt.year - (companies['Year Founded'])

### Input validation

The data has some issues with bad data, duplicate rows, and inconsistent `Industry` labels.

Identify and correct each of these issues.

#### Correcting bad data

Get descriptive statistics for the `Years To Unicorn` column.

In [21]:
# Identify and correct the issue with Years To Unicorn.

companies.describe()

,Valuation,Date Joined,Year Founded,Years to Unicorn
count,1074.000000,1074,1074.000000,1074.000000
mean,3.445996,2020-05-14 20:22:47.597765376,2012.870577,7.013035
min,1.000000,2007-07-02 00:00:00,1919.000000,-3.000000
25%,1.000000,2019-05-15 06:00:00,2011.000000,4.000000
50%,2.000000,2021-03-26 00:00:00,2014.000000,6.000000
75%,3.000000,2021-09-16 00:00:00,2016.000000,9.000000
max,180.000000,2022-04-05 00:00:00,2021.000000,98.000000
std,8.544242,NaN,5.705494,5.331842


Isolate all rows where the `Years To Unicorn` column contains a negative value.

In [23]:
# Isolate any rows where `Years To Unicorn` is negative

companies[companies['Years to Unicorn'] < 0]

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years to Unicorn
527,InVision,2,2017-11-01,Internet software & services,New York,United States,North America,2020,$349M,"FirstMark Capital, Tiger Global Management, IC...",-3


An internet search reveals that InVision was founded in 2011. Replace the value at `Year Founded` with `2011` for InVision's row. 

In [25]:
# Replace InVision's `Year Founded` value with 2011

companies.loc[527, 'Year Founded'] = 2011
# Verify the change was made properly
companies[527:528]

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years to Unicorn
527,InVision,2,2017-11-01,Internet software & services,New York,United States,North America,2011,$349M,"FirstMark Capital, Tiger Global Management, IC...",-3


Now, recalculate all the values in the `Years To Unicorn` column to remove the negative value for InVision. Verify that there are no more negative values afterwards.

In [27]:
# Recalculate all values in the `Years To Unicorn` column

companies['Years to Unicorn'] = (companies['Date Joined']).dt.year - (companies['Year Founded'])

# Verify that there are no more negative values in the column

companies['Years to Unicorn'].describe()


count    1074.000000
mean        7.021415
std         5.323155
min         0.000000
25%         4.000000
50%         6.000000
75%         9.000000
max        98.000000
Name: Years to Unicorn, dtype: float64

#### Issues with `Industry` labels

The company provided you with the following list of industry labels to identify in the data for `Industry`. 

**Note:** Any labels in the `Industry` column that are not in `industry_list` are misspellings.

In [29]:
# List provided by the company of the expected industry labels in the data
industry_list = ['Artificial intelligence', 'Other','E-commerce & direct-to-consumer', 'Fintech',\
       'Internet software & services','Supply chain, logistics, & delivery', 'Consumer & retail',\
       'Data management & analytics', 'Edtech', 'Health', 'Hardware','Auto & transportation', \
        'Travel', 'Cybersecurity','Mobile & telecommunications']

First, check if there are values in the `Industry` column that are not in `industry_list`. If so, what are they?

In [31]:
# Check which values are in `Industry` but not in `industry_list`

companies[~companies['Industry'].isin(industry_list)].groupby(by='Industry').count()[['Company']]

,Company
Industry,
Artificial Intelligence,10
Data management and analytics,6
FinTech,19


Now, correct the bad entries in the `Industry` column by replacing them with an approved string from `industry_list`.

1. Create a dictionary called `replacement_dict` whose keys are the incorrect spellings in the `Industry` series and whose values are the correct spelling, as indicated in `industry_list`.

2. Call the `replace()` method on the `Industry` series and pass to it `replacement_dict` as its argument. Reassign the result back to the `Industry` column.

3. Verify that there are no longer any elements in `Industry` that are not in `industry_list`.

In [33]:
# 1. Create `replacement_dict`

replacement_dict = {'Artificial Intelligence':'Artificial intelligence', 
                    'Data management and analytics':'Data management & analytics' , 
                    'FinTech':'Fintech'
                   }
# 2. Replace the incorrect values in the `Industry` column

companies['Industry'] = companies['Industry'].replace(replacement_dict)
# 3. Verify that there are no longer any elements in `Industry` that are not in `industry_list`

companies[~companies['Industry'].isin(industry_list)]

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years to Unicorn


#### Handling duplicate rows

The business mentioned that no company should appear in the data more than once.

Verify that this is indeed the case, and if not, clean the data so each company appears only once.

Begin by checking which, if any, companies are duplicated. Filter the data to return all occurrences of those duplicated companies.

In [35]:
# Isolate rows of all companies that have duplicates

companies[companies['Company'].duplicated(keep=False)]

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years to Unicorn
385,BrewDog,2,2017-04-10,Consumer & retail,Aberdeen,United Kingdom,Europe,2007,$233M,"TSG Consumer Partners, Crowdcube",10
386,BrewDog,2,2017-04-10,Consumer & retail,Aberdeen,UnitedKingdom,Europe,2007,$233M,TSG Consumer Partners,10
510,ZocDoc,2,2015-08-20,Health,New York,United States,North America,2007,$374M,"Founders Fund, Khosla Ventures, Goldman Sachs",8
511,ZocDoc,2,2015-08-20,Health,NaN,United States,North America,2007,$374M,Founders Fund,8
1031,SoundHound,1,2018-05-03,Artificial intelligence,Santa Clara,United States,North America,2005,$215M,"Tencent Holdings, Walden Venture Capital, Glob...",13
1032,SoundHound,1,2018-05-03,Other,Santa Clara,United States,North America,2005,$215M,Tencent Holdings,13


Keep the first occurrence of each duplicate company and drop the subsequent rows that are copies.

In [37]:
# Drop rows of duplicate companies after their first occurrence

companies = companies.drop_duplicates(subset='Company', keep='first')


### Convert numerical data to categorical data

Sometimes, you'll want to simplify a numeric column by converting it to a categorical column. To do this, one common approach is to break the range of possible values into a defined number of equally sized bins and assign each bin a name. In the next step, you'll practice this process.

#### Create a `High Valuation` column

The data in the `Valuation` column represents how much money (in billions, USD) each company is valued at. Use the `Valuation` column to create a new column called `High Valuation`. For each company, the value in this column should be `low` if the company is in the bottom 50% of company valuations and `high` if the company is in the top 50%.

In [40]:
# Create new `High Valuation` column
# Use qcut to divide Valuation into 'high' and 'low' Valuation groups

companies.insert(2, 
                 'High Valuation', 
                 pd.qcut(companies['Valuation'], 2, labels=['low', 'high'])
                )

### Convert categorical data to numerical data

Three common methods for changing categorical data to numerical are:

1. Label encoding: order matters (ordinal numeric labels)
2. Label encoding: order doesn't matter (nominal numeric labels)
3. Dummy encoding: order doesn't matter (creation of binary columns for each possible category contained in the variable)

The decision on which method to use depends on the context and must be made on a case-to-case basis. However, a distinction is typically made between categorical variables with equal weight given to all possible categories vs. variables with a hierarchical structure of importance to their possible categories.  

For example, a variable called `subject` might have possible values of `history`, `mathematics`, `literature`. In this case, each subject might be **nominal**&mdash;given the same level of importance. However, you might have another variable called `class`, whose possible values are `freshman`, `sophomore`, `junior`, `senior`. In this case, the class variable is **ordinal**&mdash;its values have an ordered, hierarchical structure of importance. 

Machine learning models typically need all data to be numeric, and they generally use ordinal label encoding (method 1) and dummy encoding (method 3). 

In the next steps, you'll convert the following variables: `Continent`, `Country/Region`, and `Industry`, each using a different approach.

### Convert `Continent` to numeric

For the purposes of this exercise, suppose that the investment group has specified that they want to give more weight to continents with fewer unicorn companies because they believe this could indicate unrealized market potential. 

Rank the continents in descending order from the greatest number of unicorn companies to the least.

In [44]:
# Rank the continents by number of unicorn companies

companies.groupby(by='Continent').count().sort_values(by='Company', ascending=False)[['Company']]

,Company
Continent,
North America,586
Asia,310
Europe,143
South America,21
Oceania,8
Africa,3


Now, create a new column called `Continent Number` that represents the `Continent` column converted to numeric in the order of their number of unicorn companies, where North America is encoded as `1`, through Africa, encoded as `6`.

In [46]:
# Create numeric `Continent Number` column

companies.insert(7,
                 'Continent Number',
                companies['Continent'].replace({'North America':1,
                'Asia':2,
                'Europe':3,
                'South America':4,
                'Oceania':5,
                'Africa':6}))

C:\Users\user\AppData\Local\Temp\ipykernel_6984\144325065.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  companies['Continent'].replace({'North America':1,


### Convert `Country/Region` to numeric

Now, suppose that within a given continent, each company's `Country/Region` is given equal importance. For analytical purposes, you want to convert the values in this column to numeric without creating a large number of dummy columns. Use label encoding of this nominal categorical variable to create a new column called `Country/Region Numeric`, wherein each unique `Country/Region` is assigned its own number. 

In [48]:
# Create `Country/Region Numeric` column
# Create numeric categories for Country/Region

companies.insert(6, 'Country/Region Numeric',companies['Country/Region'].astype('category').cat.codes)

### Convert `Industry` to numeric

Finally, create dummy variables for the values in the `Industry` column. 

In [50]:
# Convert `Industry` to numeric data
# Create dummy variables with Industry values

pd.get_dummies(companies['Industry'])
# Combine `companies` DataFrame with new dummy Industry columns

companies = pd.concat([companies, pd.get_dummies(companies['Industry'])], axis=1)

Display the first few rows of `companies`

In [52]:
companies[:10]

,Company,Valuation,High Valuation,Date Joined,Industry,City,Country/Region Numeric,Country/Region,Continent Number,Continent,Year Founded,Funding,Select Investors,Years to Unicorn,Artificial intelligence,Auto & transportation,Consumer & retail,Cybersecurity,Data management & analytics,E-commerce & direct-to-consumer,Edtech,Fintech,Hardware,Health,Internet software & services,Mobile & telecommunications,Other,"Supply chain, logistics, & delivery",Travel
0,Bytedance,180,high,2017-04-07,Artificial intelligence,Beijing,9,China,2,Asia,2012,$8B,"Sequoia Capital China, SIG Asia Investments, S...",5,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,SpaceX,100,high,2012-12-01,Other,Hawthorne,44,United States,1,North America,2002,$7B,"Founders Fund, Draper Fisher Jurvetson, Rothen...",10,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
2,SHEIN,100,high,2018-07-03,E-commerce & direct-to-consumer,Shenzhen,9,China,2,Asia,2008,$2B,"Tiger Global Management, Sequoia Capital China...",10,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
3,Stripe,95,high,2014-01-23,Fintech,San Francisco,44,United States,1,North America,2010,$2B,"Khosla Ventures, LowercaseCapital, capitalG",4,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
4,Klarna,46,high,2011-12-12,Fintech,Stockholm,38,Sweden,3,Europe,2005,$4B,"Institutional Venture Partners, Sequoia Capita...",6,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
5,Canva,40,high,2018-01-08,Internet software & services,Surry Hills,1,Australia,5,Oceania,2012,$572M,"Sequoia Capital China, Blackbird Ventures, Mat...",6,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
6,Checkout.com,40,high,2019-05-02,Fintech,London,43,United Kingdom,3,Europe,2012,$2B,"Tiger Global Management, Insight Partners, DST...",7,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
7,Instacart,39,high,2014-12-30,"Supply chain, logistics, & delivery",San Francisco,44,United States,1,North America,2012,$3B,"Khosla Ventures, Kleiner Perkins Caufield & By...",2,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
8,JUUL Labs,38,high,2017-12-20,Consumer & retail,San Francisco,44,United States,1,North America,2015,$14B,Tiger Global Management,2,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
9,Databricks,38,high,2019-02-05,Data management & analytics,San Francisco,44,United States,1,North America,2013,$3B,"Andreessen Horowitz, New Enterprise Associates...",6,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False


**Reference**

[Bhat, M.A. *Unicorn Companies*](https://www.kaggle.com/datasets/mysarahmadbhat/unicorn-companies)

